In [1]:
def github() -> str:
    """
    Returns a link to this code in my github repository
    """

    return "https://github.com/<user>/<repo>/blob/main/<filename.py>"

In [2]:
import pandas as pd

def import_yearly_data(years: list) -> pd.DataFrame:
    """
    Returns data on direct emitters for given years from the EPA data.
    """
    
    upd_df = pd.DataFrame()
    for year in years:
        url = f'https://lukashager.netlify.app/econ-481/data/ghgp_data_{year}.xlsx'
        df = pd.read_excel(url,header = 3,sheet_name = 'Direct Emitters')
        df['year'] = year
        upd_df = pd.concat([df,upd_df])
    
    
    return upd_df
import_yearly_data([2019,2020,2021])

,Facility Id,FRS Id,Facility Name,City,State,Zip Code,Address,County,Latitude,Longitude,...,Underground Coal Mines,Zinc Production,Municipal Landfills,Industrial Wastewater Treatment,Manufacture of Electric Transmission and Distribution Equipment,Industrial Waste Landfills,Is some CO2 collected on-site and used to manufacture other products and therefore not emitted from the affected manufacturing process unit(s)? (as reported under Subpart G or S),"Is some CO2 reported as emissions from the affected manufacturing process unit(s) under Subpart AA, G or P collected and transferred off-site or injected (as reported under Subpart PP)?",Does the facility employ continuous emissions monitoring?,year
0,1013701,1.100709e+11,30-30 Gas Plant,Plains,TX,79355,2300 FM 1622,YOAKUM COUNTY,33.051880,-102.887920,...,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,2021
1,1012037,1.100638e+11,50 Buttes Gas Plant,Gillette,WY,82716,3669 South Hwy 50,CAMPBELL COUNTY,43.852740,-105.777810,...,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,2021
2,1010475,NaN,AJAX PLANT,Wheeler,TX,79014,16600 CR N,NaN,35.546399,-100.116256,...,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,2021
3,1003129,1.100622e+11,AKER PLANT,STREETMAN,TX,75859,261 FCR 181,FREESTONE COUNTY,31.910000,-96.223300,...,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,2021
4,1005887,1.100706e+11,ALLIANCE COMPRESSOR STATION,JUSTIN,TX,76247,12495 HWY 114,DENTON COUNTY,33.035120,-97.331434,...,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6571,1000912,1.100004e+11,Williams,GOOSE CREEK,SC,29445,2242 BUSHY PARK ROAD,Berkeley,33.016300,-79.928500,...,NaN,NaN,NaN,NaN,NaN,NaN,N,N,Y,2019
6572,1001640,1.100712e+11,Wrenshall LNG Storage,Wrenshall,MN,55797,2301 County Road 1,CARLTON COUNTY,46.605400,-92.385280,...,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,2019
6573,1012921,1.100712e+11,Washington Court House Compressor Station,Washington Court House,OH,43160,8233 Old US 35 NW,FAYETTE COUNTY,39.594142,-83.540226,...,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,2019
6574,1006154,1.100712e+11,Waterloo Compressor Station,Waterloo,IA,50701,1508 East Schrock Road,BLACK HAWK COUNTY,42.399463,-92.327859,...,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,2019


In [3]:
pip install pyxlsb

  Using cached pyxlsb-1.0.10-py2.py3-none-any.whl.metadata (2.5 kB)
Using cached pyxlsb-1.0.10-py2.py3-none-any.whl (23 kB)
Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd

def import_parent_companies(years: list) -> pd.DataFrame:
    """
    Takes in a list of years, returns a dataframe of the parent companies excel sheets from those years
    """
    url = 'https://lukashager.netlify.app/econ-481/data/ghgp_data_parent_company_09_2023.xlsb'
    upd_df = pd.DataFrame()
    for year in years:
        df = pd.read_excel(url,sheet_name = str(year), header = 0)
        df = df.dropna(axis = 0, how = 'all')
        df['year'] = year
        upd_df = pd.concat([df,upd_df])
    return upd_df
data = import_parent_companies([2019,'2020','2021','2022'])

In [15]:
def n_null(df: pd.DataFrame, col: str) -> int:
    """
    Takes a data frame and a column name and returns the number of null values in said column
    """
    nulls = df[col].isnull().sum()
    return nulls
print(n_null(data,'FRS ID (FACILITY)'))
print(n_null(data, 'GHGRP FACILITY ID'))

2844
0


In [6]:
def clean_data(emissions_data: pd.DataFrame, parent_data: pd.DataFrame) -> pd.DataFrame:
    """
    Merges and cleans emission and parent company data by year and facility ID
    """
    merged_data = emissions_data.merge(parent_data,how = 'left',left_on=['Facility Id', 'year'],right_on = ['GHGRP FACILITY ID', 'year'])
#merged_data.columns.values.tolist()
    merged_data = merged_data[['Facility Id','year','State','Industry Type (sectors)','Total reported direct emissions','PARENT CO. STATE','PARENT CO. PERCENT OWNERSHIP']]
    merged_data.columns = merged_data.columns.str.lower()
    return merged_data

In [7]:
emissions_data = import_yearly_data(['2019','2020','2021','2022'])
parent_data = import_parent_companies(['2019','2020','2021','2022'])


In [27]:
merged_data = emissions_data.merge(parent_data,how = 'left',left_on=['Facility Id', 'year'],right_on = ['GHGRP FACILITY ID', 'year'])
merged_data = merged_data[['Facility Id','year','State','Industry Type (sectors)','Total reported direct emissions','PARENT CO. STATE','PARENT CO. PERCENT OWNERSHIP']]
merged_data.columns = merged_data.columns.str.lower()
merged_data

grouped_data = merged_data.groupby(['industry type (sectors)'])
summary_df = grouped_data[['total reported direct emissions','parent co. percent ownership']].agg([('mean','mean'),('median','median'),('min','min'),('max','max'),('sum','sum')])
sorted = summary_df.sort_values(by=('total reported direct emissions', 'sum'),ascending = False)
sorted.head(20)

total reported direct emissions  \
                                                                              mean   
industry type (sectors)                                                              
Power Plants                                                          1.253119e+06   
Petroleum and Natural Gas Systems                                     9.040685e+04   
Minerals                                                              3.018637e+05   
Chemicals                                                             3.275842e+05   
Other                                                                 9.118188e+04   
Waste                                                                 7.952563e+04   
Metals                                                                2.238148e+05   
Petroleum Product Suppliers,Refineries                                8.692042e+05   
Chemicals,Petroleum Product Suppliers,Refineries                      1.531737e+06   
Chemicals,Suppliers of CO2                                            9.492932e+05   
Pulp and Paper,Waste                                                  2.921456e+05   
Power Plants,Waste                                                    2.956812e+06   
Other,Power Plants                                                    1.939801e+06   
Chemicals,Waste                                                       1.504364e+06   
Power Plants,Suppliers of CO2                                         5.847304e+06   
Natural Gas and Natural Gas Liquids Suppliers,P...                    1.408096e+05   
Other,Waste                                                           1.382213e+05   
Metals,Waste                                                          8.441354e+05   
Chemicals,Refineries,Suppliers of CO2                                 3.210150e+06   
Pulp and Paper                                                        1.006143e+05   

                                                                             \
                                                         median         min   
industry type (sectors)                                                       
Power Plants                                         339452.992       0.000   
Petroleum and Natural Gas Systems                     45134.740       0.000   
Minerals                                              94154.616       0.000   
Chemicals                                             88398.458       0.000   
Other                                                 45077.926       0.000   
Waste                                                 54490.324       8.750   
Metals                                                75390.656       0.000   
Petroleum Product Suppliers,Refineries               490625.190       0.000   
Chemicals,Petroleum Product Suppliers,Refineries     942503.081   42122.910   
Chemicals,Suppliers of CO2                           618761.183     778.284   
Pulp and Paper,Waste                                 256818.422   19428.760   
Power Plants,Waste                                   395034.018   12604.834   
Other,Power Plants                                   298402.976       0.000   
Chemicals,Waste                                      279234.814    2211.992   
Power Plants,Suppliers of CO2                       1411841.348  816156.522   
Natural Gas and Natural Gas Liquids Suppliers,P...   106444.647       0.000   
Other,Waste                                           57012.746    9008.568   
Metals,Waste                                          69485.475    7986.798   
Chemicals,Refineries,Suppliers of CO2               3225773.322   18730.924   
Pulp and Paper                                        59302.476       0.000   

                                                                  \
                                                             max   
industry type (sectors)                                            
Power Plants                                      

In [16]:
def aggregate_emissions(df: pd.DataFrame, group_vars: list) -> pd.DataFrame:
    """
    Groups the dataframe by the given variables and outputs summary statistics for emissions and parent co. ownership
    """
    grouped_data = df.groupby(group_vars)
    summary_df = grouped_data[['total reported direct emissions','parent co. percent ownership']].agg([('mean','mean'),('median','median'),('min','min'),('max','max')])
    sorted = summary_df.sort_values(by=('total reported direct emissions', 'mean'),ascending = False)
    return sorted

In [20]:
total = aggregate_emissions(clean_data(import_yearly_data([2019,2020,2021]),import_parent_companies([2019,2020,2021])),['parent co. state'])
total

total reported direct emissions                               \
                                            mean         median           min   
parent co. state                                                                
UT                                  1.412148e+06  180794.728000     76.600000   
NM                                  1.393520e+06   78654.250000    301.748000   
ND                                  1.361010e+06  256124.436000  11642.786000   
IA                                  1.265893e+06  153483.048000    329.746000   
WY                                  1.189281e+06   64728.625000   4228.750000   
MT                                  1.054364e+06  117905.500000  19778.752000   
OR                                  9.363405e+05   53990.304000   1441.250000   
AR                                  8.192174e+05   68152.844000      0.000000   
MO                                  8.106601e+05   71898.780000      0.000000   
WI                                  8.102434e+05   86619.242000     30.800000   
GA                                  7.842243e+05   99867.250000   1903.294000   
FL                                  7.747292e+05  109018.918000      0.100000   
NE                                  7.506160e+05   62639.064000    233.300000   
OH                                  7.256200e+05   97268.624000      0.000000   
KY                                  6.665468e+05   69644.285000      0.000000   
LA                                  6.462668e+05  107243.796000   1955.544000   
NV                                  6.338314e+05  108280.987000    583.848000   
WA                                  6.109462e+05   76885.542000      0.000000   
NC                                  5.942298e+05   88263.916000    443.746000   
IN                                  5.608825e+05   69804.869000      0.000000   
CT                                  5.558903e+05  111503.424000   1746.144000   
CO                                  5.532524e+05   75236.018000      0.000000   
PR                                  5.512645e+05   85394.996000      0.000000   
AL                                  5.491216e+05  138214.875000   9870.328000   
NJ                                  5.450619e+05   95759.550000     57.700000   
MN                                  5.367577e+05   70064.864000      0.000000   
ID                                  5.145805e+05   70135.500000    200.696000   
MA                                  5.085618e+05   69727.024000    142.000000   
AZ                                  5.069432e+05   53115.950000      0.000000   
MI                                  4.915846e+05   52784.208000      0.000000   
PA                                  4.865996e+05   68020.094000      0.000000   
VA                                  4.795664e+05   58428.105000     34.500000   
WV                                  4.613487e+05   82613.890000     31.300000   
SD                                  4.482446e+05   92183.528000      0.000000   
IL                                  4.438068e+05   68267.587000      0.000000   
DC                                  3.567519e+05   65777.425000      0.000000   
AK                                  3.345794e+05   78781.494000    175.700000   
TX                                  3.281066e+05   66434.126361      0.000000   
DE                                  3.262482e+05   67054.412000      0.000000   
MS                                  3.223873e+05  180059.625000   1700.250000   
MD                                  3.115373e+05   71313.775000      0.000000   
KS                                  3.098491e+05   67680.926000      0.800000   
SC                                  2.889295e+05   58779.428000      0.000000   
VI                                  2.858286e+05  276922.007000  27926.980000   
OK                                  2.332856e+05   56092.792000      0.000000   
NY                                  2.248567e+05   47953.324000      0.000000   
CA                                  2.180415e+05